In [358]:
import numpy as np

# Генерация случайной матрицы 

In [359]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [360]:
n = 3 # size 

A = np.random.rand(n, n)
A = A@A.T

while not is_pos_def(A):
    A = np.random.rand(n, n)
    A = A@A.T
b = np.random.rand(n, 1)

eps = 1e-6 # precision 

In [361]:
A, b

(array([[0.74814869, 0.56994888, 0.1542728 ],
        [0.56994888, 0.70861236, 0.40013508],
        [0.1542728 , 0.40013508, 0.39726961]]), array([[0.18355433],
        [0.48214387],
        [0.34350191]]))

In [362]:
def f(x):
    return np.asscalar(1/2*x.T@A@x + b.T@x)

# Метод наискорейшего спуска 

In [363]:
x = b # init approximation 
i = 0 # count 

while True:
    q = A@x + b # grad 
    
    mu = q.T @ (A@x + b) / (q.T @ A @ q)
    
    x_old = x
    x = x - np.asscalar(mu) * q
    
    i += 1
    if(np.linalg.norm(x - x_old) < eps or i > 1000000):
        break

x_sd, i_sd = x, i

In [364]:
x_sd, i_sd

(array([[ 1.06923169],
        [-1.89608906],
        [ 0.62988992]]), 181)

In [365]:
f(x)

-0.25077907325830495

# Покоординатный спуск 

E - вспомогательная матрца, оттуда мы будем брать орты пространства 

In [366]:
E = np.eye(n) 
E = np.matrix(E)
E[0]

matrix([[1., 0., 0.]])

In [367]:
x = b
i = 0

while True: 
    q = E[i % n].T
    mu = q.T @ (A@x + b) / (q.T @ A @ q)
    
    x_old = x
    x = x - np.asscalar(mu) * q
    
    i += 1
    if(np.linalg.norm(x - x_old) < eps or i > 1000000):
        break
        
x_coord, i_coord = x, i

In [368]:
x_coord, i_coord

(matrix([[ 1.06922892],
         [-1.8960841 ],
         [ 0.62988727]]), 370)

In [369]:
f(x)

-0.25077907325450266

# Точный метод (метод Гаусса) 

In [370]:
def gaus(A, b):
    n = A.shape[0]
    for i in range(n):
        b[i] = b[i] / A[i][i]
        A[i] = A[i] / A[i][i]
        for j in range(i + 1, n):
            b[j] = b[j] - A[j][i] * b[i]
            A[j] = A[j] - A[j][i] * A[i]

    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            b[j] -= b[i] * A[j][i]
    return b

In [371]:
x_pr = gaus(A.copy(), -b.copy())

In [372]:
x_pr

array([[ 1.06923909],
       [-1.89610086],
       [ 0.62989981]])

In [373]:
f(x_pr)

-0.25077907326237764

# Сравнение методов 

## наискорейший спуск 

In [374]:
print("Расхождение с точным ответом: ", np.linalg.norm(x_sd - x_pr))
print("Количество шагов: ", i_sd)

Расхождение с точным ответом:  1.707979842110449e-05
Количество шагов:  181


## покоординатный спуск 

In [375]:
print("Расхождение с точным ответом: ", np.linalg.norm(x_coord - x_pr))
print("Количество шагов: ", i_coord)

Расхождение с точным ответом:  2.3270048377612103e-05
Количество шагов:  370


# Проверка необходимого условия минимума 

## точный метод

In [376]:
if(np.linalg.norm(A @ x_pr + b) < eps): 
    print("PASSED", A @ x_pr + b)
else: 
    print("NOT PASSED", A @ x_pr + b)

PASSED [[-1.11022302e-16]
 [ 1.66533454e-16]
 [ 5.55111512e-17]]


## наискорейший спуск 

In [377]:
if(np.linalg.norm(A @ x_sd + b) < eps): 
    print("PASSED", A @ x_sd + b)
else: 
    print("NOT PASSED", A @ x_sd + b)

PASSED [[-3.41569383e-07]
 [ 1.82288585e-07]
 [-3.50540003e-07]]


## покоординатный спуск 

In [378]:
if(np.linalg.norm(A @ x_pr + b) < eps): 
    print("PASSED", A @ x_pr + b)
else: 
    print("NOT PASSED", A @ x_pr + b)

PASSED [[-1.11022302e-16]
 [ 1.66533454e-16]
 [ 5.55111512e-17]]
